In [1]:
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Dropout, Flatten, Input
from keras.optimizers import RMSprop
from keras.utils.np_utils import to_categorical
from PIL import Image
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import accuracy_score
from keras import backend as K

Using TensorFlow backend.


In [2]:
#training label
labels = np.load('labels.npy')
labels.shape

(33402, 62)

In [3]:
#img as train data
size = labels.shape[0]
numsample = 5
folder = 'train/croppedsampled/'
images = []

for k in range(numsample):
    for i in range(size):
        im = Image.open(folder+str(i+1)+'_'+str(k)+'.png')
        images.append(np.asarray(im))
        
images = np.array(images)
images.shape

(167010, 54, 54, 3)

In [4]:
#replicate labels to cover all sample
labels = np.tile(labels,(5,1))
labels.shape

(167010, 62)

In [5]:
#split training set and validation set
rs = ShuffleSplit(n_splits=1, test_size=.1, random_state=0)
train_index, test_index = next(rs.split(images)) #just 1
test_images = images[test_index]
test_labels = labels[test_index]
images = images[train_index]
labels = labels[train_index]

In [2]:
#initial model (same as initial model in other program)
a = Input(shape=(54, 54, 3))
#conv layer
c1 = Conv2D(64, 3,3, activation='relu', border_mode='same')(a)
c1p = MaxPooling2D(pool_size=(2, 2)) (c1)
c1d = Dropout(0.1)(c1p)

c2 = Conv2D(128, 3,3, activation='relu', border_mode='same')(c1d)
c2p = MaxPooling2D(pool_size=(2, 2)) (c2)
c2d = Dropout(0.1)(c2p)

c3 = Conv2D(192, 3,3, activation='relu', border_mode='same')(c2d)
c3p = MaxPooling2D(pool_size=(2, 2)) (c3)
c3d = Dropout(0.1)(c3p)


c4 = Conv2D(256, 3,3, activation='relu', border_mode='same')(c3d)
c4p = MaxPooling2D(pool_size=(2, 2)) (c4)
c4d = Dropout(0.1)(c4p)


c5 = Conv2D(256, 3,3, activation='relu', border_mode='same')(c4d)
c5p = MaxPooling2D(pool_size=(2, 2)) (c5)
c5d = Dropout(0.1)(c5p)

#linear layer
flat = Flatten()(c5d)
d1 = Dense(128, activation='relu')(flat)
d1d = Dropout(0.1)(d1)

#output
o_length = Dense(7, activation='softmax')(d1d)
o_digit1 = Dense(11, activation='softmax')(d1d)
o_digit2 = Dense(11, activation='softmax')(d1d)
o_digit3 = Dense(11, activation='softmax')(d1d)
o_digit4 = Dense(11, activation='softmax')(d1d)
o_digit5 = Dense(11, activation='softmax')(d1d)
model = Model(a, [o_length,o_digit1,o_digit2,o_digit3,o_digit4,o_digit5])

rms = RMSprop(lr=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=rms)


In [3]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 54, 54, 3)     0                                            
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 54, 54, 64)    1792        input_1[0][0]                    
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 27, 27, 64)    0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 27, 27, 64)    0           maxpooling2d_1[0][0]             
___________________________________________________________________________________________

In [31]:
#training (50 epoch here)
result = model.fit(images, [labels[:,:7], labels[:,7:18], labels[:,18:29], labels[:,29:40], labels[:,40:51], labels[:,51:62]], 32, 50, validation_split=0.1, shuffle=True)

Train on 135278 samples, validate on 15031 samples
Epoch 1/50
135278/135278 [==============================] - 284s - loss: 0.9680 - dense_54_loss: 0.0882 - dense_55_loss: 0.2929 - dense_56_loss: 0.3092 - dense_57_loss: 0.1996 - dense_58_loss: 0.0738 - dense_59_loss: 0.0043 - val_loss: 0.8112 - val_dense_54_loss: 0.0865 - val_dense_55_loss: 0.2524 - val_dense_56_loss: 0.2417 - val_dense_57_loss: 0.1719 - val_dense_58_loss: 0.0566 - val_dense_59_loss: 0.0021
Epoch 2/50
135278/135278 [==============================] - 282s - loss: 0.9740 - dense_54_loss: 0.0886 - dense_55_loss: 0.2983 - dense_56_loss: 0.3065 - dense_57_loss: 0.2028 - dense_58_loss: 0.0736 - dense_59_loss: 0.0043 - val_loss: 0.8762 - val_dense_54_loss: 0.0970 - val_dense_55_loss: 0.2596 - val_dense_56_loss: 0.2658 - val_dense_57_loss: 0.1802 - val_dense_58_loss: 0.0714 - val_dense_59_loss: 0.0021
Epoch 3/50
135278/135278 [==============================] - 281s - loss: 0.9450 - dense_54_loss: 0.0866 - dense_55_loss: 0.2871

135278/135278 [==============================] - 266s - loss: 1.1155 - dense_54_loss: 0.1119 - dense_55_loss: 0.3495 - dense_56_loss: 0.3489 - dense_57_loss: 0.2231 - dense_58_loss: 0.0778 - dense_59_loss: 0.0043 - val_loss: 0.8413 - val_dense_54_loss: 0.0906 - val_dense_55_loss: 0.2715 - val_dense_56_loss: 0.2581 - val_dense_57_loss: 0.1683 - val_dense_58_loss: 0.0508 - val_dense_59_loss: 0.0021
Epoch 38/50
135278/135278 [==============================] - 266s - loss: 1.1266 - dense_54_loss: 0.1129 - dense_55_loss: 0.3523 - dense_56_loss: 0.3569 - dense_57_loss: 0.2236 - dense_58_loss: 0.0767 - dense_59_loss: 0.0043 - val_loss: 0.8572 - val_dense_54_loss: 0.0970 - val_dense_55_loss: 0.2682 - val_dense_56_loss: 0.2642 - val_dense_57_loss: 0.1694 - val_dense_58_loss: 0.0563 - val_dense_59_loss: 0.0021
Epoch 39/50
135278/135278 [==============================] - 266s - loss: 1.1280 - dense_54_loss: 0.1132 - dense_55_loss: 0.3555 - dense_56_loss: 0.3548 - dense_57_loss: 0.2227 - dense_58_

In [32]:
#validation
score = model.predict(test_images)

score = np.concatenate(score, axis=1)
#round score to get actual prediction instead of probability
roundedScore = np.zeros(score.shape, dtype="int32")
#choose threshold
roundedScore[score > 0.5] = 1

#rounded acc (exactly match)
accuracy_score(test_labels, roundedScore)

0.76103227351655589

In [6]:
#build cnn
a = Input(shape=(54, 54, 3))
#conv layer
c1 = Conv2D(64, 7,7, activation='tanh', border_mode='same')(a)
c1p = MaxPooling2D(pool_size=(2, 2)) (c1)
c1d = Dropout(0.1)(c1p)

c2 = Conv2D(128, 7,7, activation='tanh', border_mode='same')(c1d)
c2p = MaxPooling2D(pool_size=(2, 2)) (c2)
c2d = Dropout(0.1)(c2p)

c3 = Conv2D(192, 7,7, activation='tanh', border_mode='same')(c2d)
c3p = MaxPooling2D(pool_size=(2, 2)) (c3)
c3d = Dropout(0.1)(c3p)


c4 = Conv2D(256, 7,7, activation='tanh', border_mode='same')(c3d)
c4p = MaxPooling2D(pool_size=(2, 2)) (c4)
c4d = Dropout(0.1)(c4p)


c5 = Conv2D(256, 7,7, activation='tanh', border_mode='same')(c4d)
c5p = MaxPooling2D(pool_size=(2, 2)) (c5)
c5d = Dropout(0.1)(c5p)

#linear layer
flat = Flatten()(c5d)
d1 = Dense(128, activation='tanh')(flat)
d1d = Dropout(0.1)(d1)

#output
o_length = Dense(7, activation='softmax')(d1d)
o_digit1 = Dense(11, activation='softmax')(d1d)
o_digit2 = Dense(11, activation='softmax')(d1d)
o_digit3 = Dense(11, activation='softmax')(d1d)
o_digit4 = Dense(11, activation='softmax')(d1d)
o_digit5 = Dense(11, activation='softmax')(d1d)
model = Model(a, [o_length,o_digit1,o_digit2,o_digit3,o_digit4,o_digit5])

rms = RMSprop(lr=0.00005)
model.compile(loss='categorical_crossentropy', optimizer=rms)


In [8]:
result = model.fit(images, [labels[:,:7], labels[:,7:18], labels[:,18:29], labels[:,29:40], labels[:,40:51], labels[:,51:62]], 32, 50, validation_split=0.1, shuffle=True)

Train on 135278 samples, validate on 15031 samples
Epoch 1/50
135278/135278 [==============================] - 736s - loss: 4.1654 - dense_2_loss: 0.3800 - dense_3_loss: 1.3210 - dense_4_loss: 1.5122 - dense_5_loss: 0.7790 - dense_6_loss: 0.1608 - dense_7_loss: 0.0124 - val_loss: 2.7427 - val_dense_2_loss: 0.2182 - val_dense_3_loss: 0.8313 - val_dense_4_loss: 0.9927 - val_dense_5_loss: 0.5739 - val_dense_6_loss: 0.1242 - val_dense_7_loss: 0.0024
Epoch 2/50
135278/135278 [==============================] - 718s - loss: 2.3851 - dense_2_loss: 0.1968 - dense_3_loss: 0.7065 - dense_4_loss: 0.8278 - dense_5_loss: 0.5227 - dense_6_loss: 0.1278 - dense_7_loss: 0.0035 - val_loss: 1.9378 - val_dense_2_loss: 0.1544 - val_dense_3_loss: 0.5950 - val_dense_4_loss: 0.6487 - val_dense_5_loss: 0.4297 - val_dense_6_loss: 0.1085 - val_dense_7_loss: 0.0015
Epoch 3/50
135278/135278 [==============================] - 718s - loss: 1.7561 - dense_2_loss: 0.1432 - dense_3_loss: 0.5180 - dense_4_loss: 0.5869 - 

C:\Users\ryau1\Anaconda3\lib\site-packages\keras-1.2.0-py3.5.egg\keras\callbacks.py:70: UserWarning: Method on_batch_end() is slow compared to the batch update (0.150605). Check your callbacks.
  % delta_t_median)


135278/135278 [==============================] - 715s - loss: 0.1927 - dense_2_loss: 0.0160 - dense_3_loss: 0.0623 - dense_4_loss: 0.0565 - dense_5_loss: 0.0362 - dense_6_loss: 0.0209 - dense_7_loss: 8.6938e-04 - val_loss: 0.5602 - val_dense_2_loss: 0.0611 - val_dense_3_loss: 0.1630 - val_dense_4_loss: 0.1804 - val_dense_5_loss: 0.1189 - val_dense_6_loss: 0.0365 - val_dense_7_loss: 2.7982e-04
Epoch 21/50
135278/135278 [==============================] - 712s - loss: 0.1895 - dense_2_loss: 0.0158 - dense_3_loss: 0.0628 - dense_4_loss: 0.0557 - dense_5_loss: 0.0350 - dense_6_loss: 0.0194 - dense_7_loss: 8.5764e-04 - val_loss: 0.5380 - val_dense_2_loss: 0.0590 - val_dense_3_loss: 0.1584 - val_dense_4_loss: 0.1694 - val_dense_5_loss: 0.1162 - val_dense_6_loss: 0.0347 - val_dense_7_loss: 3.6448e-04
Epoch 22/50
135278/135278 [==============================] - 712s - loss: 0.1786 - dense_2_loss: 0.0138 - dense_3_loss: 0.0594 - dense_4_loss: 0.0540 - dense_5_loss: 0.0319 - dense_6_loss: 0.0185 

In [9]:
#validation
score = model.predict(test_images)

score = np.concatenate(score, axis=1)
#round score to get actual prediction instead of probability
roundedScore = np.zeros(score.shape, dtype="int32")
#choose threshold
roundedScore[score > 0.5] = 1

#rounded acc (exactly match)
accuracy_score(test_labels, roundedScore)

0.89335967906113412

In [21]:
#save model weights for evaluation
weights = model.get_weights()
np.save('final_model_weight.npy', weights)

In [ ]:
##load and use real test set to check accuracy


In [23]:
labels = np.load('test_labels.npy')
labels.shape

(13068, 62)

In [24]:
#img as train data
size = labels.shape[0]
numsample = 5
folder = 'test/croppedtight/'
images = []

for i in range(size):
    im = Image.open(folder+str(i+1)+'tight.png')
    images.append(np.asarray(im))
        
images = np.array(images)
images.shape

(13068, 54, 54, 3)

In [25]:
score = model.predict(images)

score = np.concatenate(score, axis=1)
roundedScore = np.zeros(score.shape, dtype="int32")
#choose threshold
roundedScore[score > 0.5] = 1

#rounded acc (exactly match)
accuracy_score(labels, roundedScore)

0.70684113865932052

In [39]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 54, 54, 3)     0                                            
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 54, 54, 64)    9472        input_1[0][0]                    
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 27, 27, 64)    0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 27, 27, 64)    0           maxpooling2d_1[0][0]             
___________________________________________________________________________________________